In [2]:
%matplotlib inline

In [42]:
from scipy.optimize import brentq
from tabulate import tabulate
import numpy as np

days_in_year = 364.
bonds = [
    { # the lection example
        'N': 1000.,
        'P': 1050,
        'g': 0.09,
        'p': 2.,
        'q': 45.,
        'T': 3.8,
        'n': 0,
        'external_returns': 8.1,
        'ISIN': 'lecture',
    },
    { # SoftLine ISIN RU000A0ZZZU3 https://smart-lab.ru/q/bonds/RU000A0ZZZU3 returns 9.9%
        'N': 1000.,
        'P': 1020,
        'g': 0.11,
        'p': 2.,
        'q': 0.0,
        'T': 613./days_in_year,
        'n': 0.0,
        'external_returns': 9.9,
        'ISIN': 'RU000A0ZZZU3',
    },
    { # Норильский никель ISIN RU000A100VQ6 https://smart-lab.ru/q/bonds/RU000A100VQ6 returns 6.79% 
        'N': 1000.,
        'P': 1009.7,
        'g': 0.072,
        'p': 2,
        'q': 0.0,
        'T': 1620./days_in_year,
        'n': 0.0,
        'external_returns': 6.79,
        'ISIN': 'RU000A100VQ6',
    }
]

def fill_in_derivatives(bond):
    bond['q'] = bond['g'] * bond['N'] / bond['p']
    bond['n'] = int(bond['T'] * bond['p'])
    if bond['T'] * bond['p'] - bond['n'] != 0:
        bond['n'] += 1
        

for bond in bonds:
    fill_in_derivatives(bond)


def calculate_irr(x):
    n = bond['n']
    tau = n / bond['p'] - bond['T']

    return bond['P'] - (1 + x)**tau * (bond['g'] * bond['N'] / bond['p']\
            * (1 - (1 + x)**(-n/bond['p']))/((1 + x)**(1/bond['p']) - 1) + bond['N'] / (1 + x)**(n / bond['p']))


def calculate_pv(cf_i: float,
                 returns: float,
                 t_i: float):
    return cf_i/(1+returns)**t_i


def calculate_dur():
    yield 0.0


def calculate_convex_coef():
    yield 0.0

    
def print_calculation_table(bond: dict):
    P = bond['P']
    returns = bond['g']
    N = bond['N']
    
    CF_i = returns * N
    delta_t = 1. / bond['p']

    tabulate_rows = []
    tabulate_headers = ['№','t_i', 'CF_i', 'PV(CF_i)', 'PV(CF_i)/P', 'PV(CF_i)/P*t_i', 't_i*(t_i+1)*PV(CF_i)/P']
    for i in range(1, bond['n']+1):
        t_i = delta_t * i
        if i == bond['n']:
            CF_i += N
        PV_CF_i = calculate_pv(CF_i, returns, t_i)
        tabulate_rows.append([i, 
                              t_i, 
                              CF_i, 
                              PV_CF_i, 
                              PV_CF_i / P,
                              PV_CF_i / P * t_i, 
                              PV_CF_i / P * t_i * (t_i +1)])

    sum_row_initial = [None, None, None] + list(np.sum(tabulate_rows, axis=0, dtype=float)[3:])    
    tabulate_rows.append(sum_row_initial)
    print(tabulate(tabulate_rows,
             headers=tabulate_headers))
    print("\n")

# Ищем корень нелинейного уравнения
for bond in bonds:
    root = brentq(calculate_irr, 0.01, 0.2)
    print(f"ISIN {bond['ISIN']} Доверенное значение {bond['external_returns']}% расчётное значение {root*100:.3f}%")
    print_calculation_table(bond)
    


ISIN lecture Доверенное значение 8.1% расчётное значение 8.160%
  №    t_i    CF_i    PV(CF_i)    PV(CF_i)/P    PV(CF_i)/P*t_i    t_i*(t_i+1)*PV(CF_i)/P
---  -----  ------  ----------  ------------  ----------------  ------------------------
  1    0.5      90     86.2044     0.0820994         0.0410497                 0.0615745
  2    1        90     82.5688     0.078637          0.078637                  0.157274
  3    1.5      90     79.0866     0.0753205         0.112981                  0.282452
  4    2        90     75.7512     0.072144          0.144288                  0.432864
  5    2.5      90     72.5565     0.0691014         0.172754                  0.604637
  6    3        90     69.4965     0.0661872         0.198561                  0.794246
  7    3.5      90     66.5656     0.0633958         0.221885                  0.998484
  8    4      1090    772.183      0.735413          2.94165                  14.7083
                     1304.41       1.2423            3.